# Data Preparation

In [1]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install tensorflow
!pip install opendatasets
!pip install opencv-python pillow

^C


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import opendatasets as ds
import os
import cv2
from PIL import Image, ImageEnhance, ImageOps
from pathlib import Path
import shutil
import zipfile
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
url = "https://www.kaggle.com/datasets/lprdosmil/unsplash-random-images-collection"
data = ds.download(url)

In [ ]:
print(os.listdir(data))

In [ ]:
input_folder = 'Filter Identification/unsplash-random-images-collection/unsplash-random-images-collection'  
output_folder = 'filtered_images_output'  

## Sepia Filter

In [ ]:
def apply_sepia(image):
    sepia_filter = np.array([[0.272, 0.534, 0.131],
                             [0.349, 0.686, 0.168],
                             [0.393, 0.769, 0.189]])
    sepia_img = cv2.transform(image, sepia_filter)
    return np.clip(sepia_img, 0, 255).astype(np.uint8)


## Black and White (B&W)

In [ ]:
def apply_black_and_white(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

## Vintage Filter

In [ ]:
def apply_vintage(image):
    rows, cols = image.shape[:2]
    vignette_filter = np.zeros((rows, cols), np.uint8)
    for i in range(rows):
        for j in range(cols):
            vignette_filter[i, j] = 255 * (1 - ((i / rows) ** 2 + (j / cols) ** 2) ** 0.5)
    vintage_img = cv2.applyColorMap(image, cv2.COLORMAP_PINK)
    return cv2.addWeighted(image, 0.7, vintage_img, 0.3, 0)


## Gaussian Blur

In [ ]:
def apply_gaussian_blur(image):
    return cv2.GaussianBlur(image, (15, 15), 0)

## Saturation Boost

In [ ]:
def apply_saturation_boost(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_img[:, :, 1] = cv2.add(hsv_img[:, :, 1], 50)
    return cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)


## Vivid Filter

In [ ]:
def apply_vivid(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_img[:, :, 1] = cv2.add(hsv_img[:, :, 1], 75)  # Increase saturation more
    return cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)


## Warm Filter

In [ ]:
def apply_warm(image):
    b, g, r = cv2.split(image)
    r = cv2.add(r, 30)  # Add warmth to the red channel
    return cv2.merge((b, g, r))

## Cool Filter

In [ ]:
def apply_cool(image):
    b, g, r = cv2.split(image)
    b = cv2.add(b, 30)  # Add coolness to the blue channel
    return cv2.merge((b, g, r))


## Soft Focus Filter

In [ ]:
def apply_soft_focus(image):
    blurred = cv2.GaussianBlur(image, (21, 21), 0)
    return cv2.addWeighted(image, 0.5, blurred, 0.5, 0)


## Film Grain Filter

In [ ]:
def apply_film_grain(image):
    noise = np.random.normal(0, 25, image.shape).astype(np.uint8)  # Add noise
    return cv2.add(image, noise)


## Cartoon Filter

In [ ]:
def apply_cartoon(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255,
                                   cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(image, 9, 300, 300)
    return cv2.bitwise_and(color, color, mask=edges)


## Dramatic Filter

In [ ]:
def apply_dramatic(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)


## Apply Filters and Save Images in Separate Folders


In [ ]:
# Function to apply filter and save image
def apply_and_save_filter(image, filter_func, filter_name, img_name, output_folder):
    filter_folder = os.path.join(output_folder, filter_name)
    Path(filter_folder).mkdir(parents=True, exist_ok=True)

    filtered_img = filter_func(image)
    output_path = os.path.join(filter_folder, img_name)
    cv2.imwrite(output_path, filtered_img)


## Function to process images in a folder

In [ ]:
def process_images(input_folder, output_folder):
    filters = {
        "Sepia": apply_sepia,
        "Black and White (B&W)": apply_black_and_white,
        "Vintage": apply_vintage,
        "Gaussian Blur": apply_gaussian_blur,
        "Saturation Boost": apply_saturation_boost,
        "Vivid": apply_vivid,
        "Warm": apply_warm,
        "Cool": apply_cool,
        "Soft Focus": apply_soft_focus,
        "Film Grain": apply_film_grain,
        "Cartoon": apply_cartoon,
        "Dramatic": apply_dramatic
    }

    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)

        if image is not None:
            for filter_name, filter_func in filters.items():
                apply_and_save_filter(image, filter_func, filter_name, img_name, output_folder)
        else:
            print(f"Could not read image {img_name}")


## Define your dataset and output paths

In [ ]:

input_folder = 'C:/Users/moksh/OneDrive/Desktop/Filter Identification/unsplash-random-images-collection/unsplash-images-collection'  # Replace with your path
output_folder = 'C:/Users/moksh/OneDrive/Desktop/Filter Identification/Filtered dataset'  


## Apply the filters and save the processed images

In [ ]:
process_images(input_folder, output_folder)

In [ ]:
print(os.listdir(data))

# Model Development

## Data Load

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import requests

In [ ]:
# Define paths
data_dir = 'C:/Users/moksh/OneDrive/Desktop/Filter Identification/Filtered dataset'  

# Set up parameters
image_size = (150, 150)  # Adjust according to your needs
batch_size = 32

# Create an ImageDataGenerator for loading images
datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load the data from the directory
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'sparse_categorical' if labels are integers
    shuffle=True
)


## Build the Model

In [ ]:
# Build the model
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Train the model

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10  
)


## Model Saving

In [ ]:
project_folder ='C:/Users/moksh/OneDrive/Desktop/Filter Identification/models'  


model_save_path = os.path.join(project_folder, 'filter_classification_model.h5')

# Save the model
model.save(model_save_path)

print(f"Model saved at: {model_save_path}")

# Testing With Image

In [ ]:
model = tf.keras.models.load_model('filter_classification_model.h5')

In [ ]:
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
def predict_image(image_path):
    # Load and preprocess the image
    image = load_and_preprocess_image(image_path)
    
    # Make the prediction
    predictions = model.predict(image)
    
    # Get the class index with the highest probability
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    
    # Get the class labels
    class_labels = list(train_generator.class_indices.keys())
    
    # Return the predicted label
    return class_labels[predicted_class_index]


## Testing  

In [ ]:
test_image_path = 'C:/Users/moksh/OneDrive/Desktop/Filter Identification/Testing images/images.jpg'

# Get the predicted label
predicted_label = predict_image(test_image_path)
print(f'The predicted label for the image is: {predicted_label}')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
data_dir = 'C:/Users/moksh/OneDrive/Desktop/Filter Identification/Filtered dataset'  # Replace with your actual path

# Parameters
image_size = (150, 150)  # Adjust as per your model's input size
batch_size = 32

# Data Generator with validation split (80% train, 20% test)
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

# Training Data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # This is the training set
)

# Validation/Test Data
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # This is the test set
)


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


In [ ]:
import numpy as np

# Get a random batch from the test set
x_test, y_test = next(test_generator)

# Make predictions
predictions = model.predict(x_test)

# Convert predictions and true labels to readable format
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

# Get class labels
class_labels = list(train_generator.class_indices.keys())

# Display results for a random image from the batch
import random
i = random.randint(0, batch_size - 1)
print(f"Predicted Label: {class_labels[predicted_labels[i]]}")
print(f"True Label: {class_labels[true_labels[i]]}")
